In [1]:
from basketball.common.team import TeamClassifier

cf = TeamClassifier()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# open images in bball
from ultralytics import YOLO
import cv2

model = YOLO("weights.pt")


In [3]:
from pathlib import Path

train_dir = Path("boxball-roblox/train/images/")

In [ ]:
crops = []
player_label = 1

for img_path in train_dir.glob("youtube-1_*"):
    if img_path.is_file():
        img = cv2.imread(str(img_path))
        result = model.predict(conf=0.8, source=str(img_path))
        
        if result[0].boxes:
            boxes = result[0].boxes.xyxy
            class_ids = result[0].boxes.cls
            
            for i, box in enumerate(boxes):
                if class_ids[i].item() == player_label:
                    x1, y1, x2, y2 = map(int, box[:4])
                    crop = img[y1:y2, x1:x2]
                    
                    if crop.size > 0:
                        crops.append(crop)
                        cv2.imwrite(f"crop_outputs/cropped_image_{i}.jpg", crop)


image 1/1 /Users/anishthewizard/Documents/Programming/Zephyr_Basketball/boxball-roblox/train/images/youtube-1_jpg.rf.53be10ad76aab911ad2f8f0ac56c30da.jpg: 640x640 1 Player, 113.7ms
Speed: 4.1ms preprocess, 113.7ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/anishthewizard/Documents/Programming/Zephyr_Basketball/boxball-roblox/train/images/youtube-1_jpg.rf.4f831ef6d5f16b830777c573e6fbfb8c.jpg: 640x640 1 Ball, 10 Players, 2 Refs, 66.0ms
Speed: 0.9ms preprocess, 66.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/anishthewizard/Documents/Programming/Zephyr_Basketball/boxball-roblox/train/images/youtube-1_jpg.rf.de0df1dc7255db78c498ca53042a223f.jpg: 640x640 1 Ball, 10 Players, 2 Refs, 54.8ms
Speed: 0.8ms preprocess, 54.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/anishthewizard/Documents/Programming/Zephyr_Basketball/boxball-roblox/train/images/youtube-1_jpg.rf.c2f83f8b2e

In [5]:
for i in range(0, len(crops), 500):
    batch_crops = crops[i:i+500]
    
    if len(batch_crops) > 0:
        cf.fit(batch_crops)
        print(f"Trained on {i + len(batch_crops)} crops, exporting model...")
        cf.save_weights(f"classifiers/iter{i}")

Embedding extraction: 5it [00:07,  1.42s/it]
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Trained on 143 crops, exporting model...


In [14]:
cf.predict([crops[5], crops[1]])

Embedding extraction: 1it [00:00,  6.61it/s]


array([1, 0], dtype=int32)

In [15]:
cv2.imshow("crop", crops[5])
cv2.waitKey(0)
cv2.imshow("crop", crops[1])
cv2.waitKey(5)

cv2.destroyAllWindows()